In [54]:
import tkinter as tk
import random
import time
root = tk.Tk()

"""CREATES A GRID,WITH A RANDOM PATH FROM STARTING POINT TO TARGET EACH TIME THE PROGRAM IS RUN,WHICH THE BALL FOLLOWS 
TO GET FROM STARTING POINT TO TARGET.
NOTE:  KEEP WIDTH AND HEIGHT SAME FOR THE CODE TO WORK PROPERLY, ALSO KEEP PARITIONS TO BE MULTIPLE OF WIDTH AND HEIGHT"""

CANVAS_WIDTH = CANVAS_HEIGHT = 400
PARTITIONS = 20


def random_start_point(canvas, line_num, line_spacing):
    """Initializes a random starting square in the first row of canvas"""
    square_num = random.randint(0, line_num - 1)
    start_point = canvas.create_rectangle(
        square_num * line_spacing,
        0,
        (square_num + 1) * line_spacing,
        line_spacing,
        fill="red",
    )
    root.update()
    left_x, top_y, right_x, bottom_y = canvas.coords(start_point)
    right_x, bottom_y = (square_num + 1) * line_spacing, line_spacing
    return left_x, right_x, top_y, bottom_y


def random_end_point(canvas, line_num, line_spacing):
    """Initializes a random tartget square in the last row of canvas"""
    square_num = random.randint(0, line_num - 1)
    target_point = canvas.create_rectangle(
        square_num * line_spacing,
        (line_num - 1) * line_spacing,
        (square_num + 1) * line_spacing,
        (line_num) * line_spacing,
        fill="green",
    )
    root.update()
    left_x, top_y, right_x, bottom_y = canvas.coords(target_point)
    right_x, bottom_y = (square_num + 1) * line_spacing, (line_num) * line_spacing
    return left_x, right_x, top_y, bottom_y


def left(left_x, choices):
    """Add Left as a choice to the list of choices if the current position
    has pixels to travel on the left side"""
    if left_x > 0:
        choices.append("Left")
    return choices


def right(right_x, CANVAS_WIDTH, choices):
    """Add Right as a choice to the list of choices if the current position
    has pixels to travel on the right side"""
    if right_x < CANVAS_WIDTH:
        choices.append("Right")
    return choices


def bottom(bottom_y, CANVAS_HEIGHT, choices):
    """Add Bottom as a choice to the list of choices if the current position
    has pixels to travel on the bottom side"""
    if bottom_y < CANVAS_HEIGHT:
        choices.append("Bottom")
    return choices


def not_move_right(sequential_choices, choices_available):
    """To not retrace steps, i.e. if we recently moved to right then don't 
    move to left, or if the one before the most recent one was right then
    don't move to left"""
    if len(sequential_choices) == 1:
        if sequential_choices[-1] == "Right":
            choices_available.remove("Left")
    elif len(sequential_choices) > 1:
        if (sequential_choices[-2] == "Right") or (sequential_choices[-1] == "Right"):
            choices_available.remove("Left")
    return choices_available


def not_move_left(sequential_choices, choices_available):
    """To not retrace steps, i.e. if we recently moved to left then don't
    move to right, or if the one before the most recent one was left then
    don't move to right"""
    if len(sequential_choices) == 1:
        if sequential_choices[-1] == "Left":
            choices_available.remove("Right")
    elif len(sequential_choices) > 1:
        if (sequential_choices[-2] == "Left") or (sequential_choices[-1] == "Left"):
            choices_available.remove("Right")
    return choices_available


def remove_left(sequential_choices, choices_available):
    """Calls the function that removes left if it is in the available choices"""
    if ("Left" in choices_available) or (sequential_choices[-1] == "Left"):
        choices_available = not_move_right(sequential_choices, choices_available)
    return choices_available


def remove_right(sequential_choices, choices_available):
    """Calls the function that removes left if it is in the available choices"""
    if ("Right" in choices_available) or (sequential_choices[-1] == "Right"):
        choices_available = not_move_left(sequential_choices, choices_available)
    return choices_available


def choice(
    left_x, right_x, top_y, bottom_y, CANVAS_HEIGHT, CANVAS_WIDTH, sequential_choices
):
    """Decides the sequential direction while building the path,
    tackles retracing problems via supporting functions, updates the 
    list that stores all the steps taken sequentially"""
    choices = []
    choices = left(left_x, choices)
    choices = right(right_x, CANVAS_WIDTH, choices)
    choices = bottom(bottom_y, CANVAS_HEIGHT, choices)
    if len(sequential_choices) > 0:
        choices = remove_right(sequential_choices, choices)
        choices = remove_left(sequential_choices, choices)
    if len(sequential_choices) == 0:
        random_choice = "Bottom"
    else:
        random_choice = random.choice(choices)
    sequential_choices.append(random_choice)
    return random_choice, sequential_choices


def move(left_x, right_x, top_y, bottom_y, choices, line_spacing, canvas):
    """Makes the path according to the choice made"""
    if choices == "Right":
        left_x += line_spacing
        right_x += line_spacing
        rect_right = canvas.create_rectangle(
            left_x, top_y, right_x, bottom_y, fill="white", outline="white"
        )
        root.update()
    elif choices == "Left":
        left_x -= line_spacing
        right_x -= line_spacing
        rect_left = canvas.create_rectangle(
            left_x, top_y, right_x, bottom_y, fill="white",outline='white'
        )
        root.update()
    elif choices == "Bottom":
        top_y += line_spacing
        bottom_y += line_spacing
        rect_bottom = canvas.create_rectangle(
            left_x, top_y, right_x, bottom_y, fill="white",outline='white'
        )
        root.update()
    return left_x, right_x, top_y, bottom_y


def move_to_target(
    path_track,
    left_x,
    right_x,
    top_y,
    bottom_y,
    target_left_x,
    target_right_x,
    target_top_y,
    target_bottom_y,
    canvas,
    line_spacing,
):
    """The path is made upto second last row, so now if the the last step is
    directly above the target stop, if not take a step down then see if the
    step is on left or right, then take steps accordingly until we get to target."""
    if (bottom_y == target_top_y) and (left_x == target_left_x):
        return
    else:
        left_x, right_x, top_y, bottom_y = move(
            left_x, right_x, top_y, bottom_y, "Bottom", line_spacing, canvas
        )
        path_track.append([left_x, top_y, right_x, bottom_y])

        if (left_x > target_right_x) and left_x != target_right_x:
            while left_x > target_right_x:
                left_x, right_x, top_y, bottom_y = move(
                    left_x, right_x, top_y, bottom_y, "Left", line_spacing, canvas
                )
                path_track.append([left_x, top_y, right_x, bottom_y])

        elif (right_x < target_left_x) and (right_x != target_left_x):
            while right_x < target_left_x:
                left_x, right_x, top_y, bottom_y = move(
                    left_x, right_x, top_y, bottom_y, "Right", line_spacing, canvas
                )
                path_track.append([left_x, top_y, right_x, bottom_y])
    return path_track


def falling_ball_animation(canvas, path):
    """Creates animation of ball following the path to get to the target,
    also the ball alternates between colors."""
    
    start_coord = path[0]
    circle = canvas.create_oval(
        start_coord[0], start_coord[1], start_coord[2], start_coord[3], fill="black"
    )
    root.update()
    colors = [
        "orange",
        "red",
        "yellow",
        "salmon",
        "cyan",
        "green",
        "purple",
        "pink",
        "magenta",
        "blue",
    ]
    for x in range(len(path) - 1):
        start_coord = path[x + 1]
        canvas.moveto(circle, start_coord[0], start_coord[1])
        color = random.choice(colors)
        canvas.itemconfig(circle, fill=color)
        root.update()
        time.sleep(0.1)


def main():
    """Creates canvas, makes a grid on it, and uses all supporting functions to create the animation"""
    canvas = tk.Canvas(root, width=CANVAS_WIDTH, height=CANVAS_HEIGHT)
    canvas.pack()
    line_num = int(CANVAS_HEIGHT / PARTITIONS - 1)
    line_spacing = CANVAS_HEIGHT / PARTITIONS
    for x in range(line_num):
        verticle_lines = canvas.create_line(
            line_spacing * (x + 1), 0, line_spacing * (x + 1), CANVAS_HEIGHT
        )
        root.update()
        horizontal_lines = canvas.create_line(
            0, line_spacing * (x + 1), CANVAS_WIDTH, line_spacing * (x + 1)
        )
        root.update()
    left_x, right_x, top_y, bottom_y = random_start_point(
        canvas, PARTITIONS, line_spacing
    )
    target_left_x, target_right_x, target_top_y, target_bottom_y = random_end_point(
        canvas, PARTITIONS, line_spacing
    )
    sequential_choices = []
    path_track = [[left_x, top_y, right_x, bottom_y]]
    while bottom_y < CANVAS_HEIGHT - line_spacing:
        choices, sequential_choices = choice(
            left_x,
            right_x,
            top_y,
            bottom_y,
            CANVAS_HEIGHT,
            CANVAS_WIDTH,
            sequential_choices,
        )
        left_x, right_x, top_y, bottom_y = move(
            left_x, right_x, top_y, bottom_y, choices, line_spacing, canvas
        )
        path_track.append([left_x, top_y, right_x, bottom_y]) #keeps track of the path created for the ball to follow
    path_track = move_to_target(
        path_track,
        left_x,
        right_x,
        top_y,
        bottom_y,
        target_left_x,
        target_right_x,
        target_top_y,
        target_bottom_y,
        canvas,
        line_spacing,
    )
    path_track.append([target_left_x, target_top_y, target_right_x, target_bottom_y]) 
    falling_ball_animation(canvas, path_track)
    root.mainloop()


if __name__ == "__main__":
    main()
